# 2 Data wrangling<a id='2_Data_wrangling'></a>

### 2.1 Importing Necessary Modules and Data

In [2]:
import pandas as pd
import os.path
from os import path

import warnings
warnings.filterwarnings("ignore")

### 2.2 Creating Features

Our data consists of 10 sensor readings place around volcanos. These 10 sensors have been normalized and separated into their own CSV file based on the volcano they were reading. The target data has been provided in a separate CSV file that links the segment ID(the name of the file our features are in) and the time to next eruption. Due to the size of our data(14GB) and the way the data is organized, we will create new features to consolidate as much data as we can into a smaller space and something that is better able to be used as an input for our models.

The raw data cannot be attached to this project but can be found at https://www.kaggle.com/c/predict-volcanic-eruptions-ingv-oe/data?select=train.csv
Place the train.csv and train folder in the predict-volcanic-eruptions folder but most of the project will focus on the Data.csv attached

In [3]:
targetData = pd.read_csv('predict-volcanic-eruptions/train.csv')

In [4]:
if not path.exists('predict-volcanic-eruptions/Data.csv'):
    columns = ['segment_id']
    featureFunctionSuffix = ['_mean','_01percentile','_05percentile','_10percentile','_20percentile','_25percentile','_35percentile','_45percentile','_65percentile','_75percentile','_80percentile','_90percentile','_99percentile','_95percentile','_std','_var','_max','_min','_range','_kurtosis','_skew','_sum','_meanAbsDev','_localExtremaCount']
   
    for i in os.listdir('predict-volcanic-eruptions/train/') :
        rows = []
        fileNameSplit = i.split('.')
        
        #Checks to make sure it only reads csv files in the folder since jupyter creates a checkpoint folder
        if fileNameSplit[1] != 'csv' :
            continue
        
        #Separates out the segmentID out of the filename to use with train.csv
        segmentID = fileNameSplit[0]
        
        #Appends the segmentID at the start of each row to use to merge
        rows.append(segmentID)
        
        #load in the raw sensor data
        rawData = pd.read_csv('predict-volcanic-eruptions/train/' + i)
        rawData = rawData.astype('float32')
    
        #Calculate the column names with the suffic sensor_#_suffix if the columns list is empty
        if columns == ['segment_id']:
            for j in rawData.columns:
                for k in featureFunctionSuffix:
                    columns.append(j + k)
                    
            #Creates a temporary dataframe with the column names just to store all the rows o
            featureData = pd.DataFrame(columns = columns)
                    
    
        #Cleaning and creating features of the data
        for j in rawData.columns:
            #Creating a temporary dataframe of just a single sensor
            columnData = rawData[[j]]
            
            #Fill in missing values. Takes average of above and below value. If consecutive nulls exist, back fill and forward fill to caputre those
            #Afterwards, fill the rest with 0, which should only be sensors that are completely missing
            columnData = ((columnData.bfill(axis=1) + columnData.ffill(axis=1)) / 2)
            columnData = columnData.bfill(axis=1).ffill(axis=1)
            
            columnData.fillna(0,inplace = True,axis = 1)
            
            #Creating features for a single sensor
            meanData = columnData.mean()
            percentile01Data = columnData.quantile(0.01)
            percentile05Data = columnData.quantile(0.05)
            percentile25Data = columnData.quantile(0.25)
            percentile75Data = columnData.quantile(0.75)
            percentile10Data = columnData.quantile(0.10)
            percentile20Data = columnData.quantile(0.20)
            percentile35Data = columnData.quantile(0.35)
            percentile45Data = columnData.quantile(0.45)
            percentile65Data = columnData.quantile(0.65)
            percentile80Data = columnData.quantile(0.80)
            percentile90Data = columnData.quantile(0.90)
            percentile99Data = columnData.quantile(0.99)
            percentile95Data = columnData.quantile(0.95)
            kurtosisData = columnData.kurtosis()
            stdData = columnData.std()
            maxData = columnData.max()
            minData = columnData.min()
            rangeData = maxData - minData
            skewData = columnData.skew()
            sumData = columnData.sum()
            varData = columnData.var()
            madData = columnData.mad()
            localMinData = columnData[(columnData.shift(1) > columnData) & (columnData.shift(-1) > columnData)].count()
            localMaxData = columnData[(columnData.shift(1) < columnData) & (columnData.shift(-1) < columnData)].count()
            
            rows.append(meanData[0])
            rows.append(percentile01Data[0])
            rows.append(percentile05Data[0])
            rows.append(percentile10Data[0])
            rows.append(percentile20Data[0])
            rows.append(percentile25Data[0])
            rows.append(percentile35Data[0])
            rows.append(percentile45Data[0])
            rows.append(percentile65Data[0])
            rows.append(percentile75Data[0])
            rows.append(percentile80Data[0])
            rows.append(percentile90Data[0])
            rows.append(percentile95Data[0])
            rows.append(percentile99Data[0])
            rows.append(stdData[0])
            rows.append(varData[0])
            rows.append(maxData[0])
            rows.append(minData[0])
            rows.append(rangeData[0])
            rows.append(kurtosisData[0])
            rows.append(skewData[0])
            rows.append(sumData[0])
            rows.append(madData[0])
            rows.append(localMinData[0] + localMaxData[0])
        
        featureData.loc[len(featureData)] = rows
    
    #Merging the feature and target data into a full single dataframe
    featureData['segment_id'] = featureData['segment_id'].astype('int64')
    fullData = featureData.merge(targetData,on='segment_id')
    
    #Exporting to csv for future use
    fullData.to_csv('predict-volcanic-eruptions/Data.csv',index = False)

We have create our own features consisting of some basic aggregate functions, such as mean, median and range. As well as others such as percentiles at many different locations, the number of peaks in each file. After turning all our data into features, we have 242 columns. 240 columns are features and 1 column target and 1 column index with 4431 rows of data.

In [6]:
data = pd.read_csv('predict-volcanic-eruptions/Data.csv')
data.head()

,segment_id,sensor_1_mean,sensor_1_01percentile,sensor_1_05percentile,sensor_1_10percentile,sensor_1_20percentile,sensor_1_25percentile,sensor_1_35percentile,sensor_1_45percentile,sensor_1_65percentile,...,sensor_10_var,sensor_10_max,sensor_10_min,sensor_10_range,sensor_10_kurtosis,sensor_10_skew,sensor_10_sum,sensor_10_meanAbsDev,sensor_10_localExtremaCount,time_to_eruption
0,1000015382,0.382244,-277.0,-174.0,-130.0,-83.0,-66.0,-38.0,-12.0,37.0,...,6.201152e+04,3179.0,-2961.0,6140.0,14.978788,0.058227,53806.0,163.679382,16748,16258654
1,1000554676,-3.828120,-1252.0,-878.0,-686.0,-446.0,-356.0,-206.0,-67.0,199.0,...,1.112280e+06,4442.0,-4329.0,8771.0,0.160791,0.004739,-445008.0,835.125977,5452,6347792
2,1000745424,8.291928,-1392.0,-989.0,-765.0,-497.0,-400.0,-230.0,-74.0,234.0,...,1.510302e+06,5230.0,-5040.0,10270.0,0.193508,-0.025480,-89519.0,972.496460,4705,5120693
3,1001461087,2.071582,-1017.0,-645.0,-485.0,-310.0,-246.0,-141.0,-46.0,140.0,...,5.420066e+05,5788.0,-4634.0,10422.0,2.732420,-0.051502,-82408.0,548.506348,9983,10393161
4,1001732002,0.904102,-702.0,-465.0,-358.0,-233.0,-187.0,-104.0,-33.0,105.0,...,8.239711e+05,4574.0,-3909.0,8483.0,1.135692,0.375558,1922895.0,691.087891,5633,20549733


In [7]:
data.describe()

,segment_id,sensor_1_mean,sensor_1_01percentile,sensor_1_05percentile,sensor_1_10percentile,sensor_1_20percentile,sensor_1_25percentile,sensor_1_35percentile,sensor_1_45percentile,sensor_1_65percentile,...,sensor_10_var,sensor_10_max,sensor_10_min,sensor_10_range,sensor_10_kurtosis,sensor_10_skew,sensor_10_sum,sensor_10_meanAbsDev,sensor_10_localExtremaCount,time_to_eruption
count,4.431000e+03,4431.000000,4431.000000,4431.000000,4431.000000,4431.000000,4431.000000,4431.000000,4431.000000,4431.000000,...,4.431000e+03,4431.00000,4431.000000,4431.000000,4431.000000,4431.000000,4.431000e+03,4431.000000,4431.000000,4.431000e+03
mean,1.074694e+09,-0.525764,-1578.586098,-1049.330399,-802.620853,-517.872038,-413.008576,-234.490860,-75.526969,233.872489,...,4.415325e+06,5228.43963,-5248.341458,10476.781088,2.611002,-0.001246,-1.970177e+04,816.007929,9427.109682,2.284891e+07
std,6.161966e+08,17.911303,3487.337427,2523.768215,2010.706912,1298.873781,1033.344497,587.431979,189.365386,585.034348,...,2.919145e+07,5172.18309,5208.346859,10342.068127,8.756652,0.123310,2.005872e+06,1467.473921,6091.334977,1.348439e+07
min,5.131810e+05,-595.469238,-32767.000000,-32767.000000,-32767.000000,-21859.000000,-17308.000000,-9715.000000,-3040.000000,0.000000,...,0.000000e+00,0.00000,-32767.000000,0.000000,-1.402664,-2.246147,-6.419482e+07,0.000000,0.000000,6.250000e+03
25%,5.527934e+08,-1.788295,-1302.500000,-883.000000,-675.000000,-435.500000,-347.000000,-197.000000,-64.000000,97.000000,...,3.380386e+05,3111.00000,-5159.000000,6257.000000,0.303863,-0.035698,-1.987590e+05,438.351517,5307.000000,1.127016e+07
50%,1.066153e+09,0.017700,-880.000000,-579.000000,-442.000000,-286.000000,-229.000000,-129.000000,-41.000000,129.000000,...,5.211669e+05,4074.00000,-4115.000000,8235.000000,0.793848,-0.000981,5.861000e+03,554.192200,6700.000000,2.246559e+07
75%,1.606350e+09,1.825470,-673.000000,-439.000000,-334.000000,-216.000000,-172.000000,-97.000000,-31.000000,197.000000,...,9.545048e+05,5161.00000,-3097.500000,10252.500000,2.360290,0.033547,2.270125e+05,753.623199,11804.500000,3.434356e+07
max,2.146939e+09,341.219940,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9772.000000,...,5.496819e+08,32767.00000,0.000000,65534.000000,234.337540,1.775161,6.134024e+07,20721.322266,34815.000000,4.904609e+07


We accounted for null values during the creation of our features and looking at the above table confirms that we have data in all our columns. Our data has been compiled into the Data.csv and this is what we will use for the rest of the capstone project to perform our EDA and Modeling on.